In [1]:
import ROOT
import numpy as np

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x7a24b00


In [2]:
%jsroot on

In [3]:
# FIXME: don't touch! the fortran code has a global state :(
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.gSystem.Load("OO/diehl/libObservable.so")

0

In [5]:
ROOT.gInterpreter.Declare("#include \"OO/diehl/OOTools_old.h\"")

True

In [6]:
# use same masses as in whizard!
mW = 80.419
mZ = 91.1882

# ROOT.OOTools.init(250, mW, mZ)
ROOT.OOTools.init(190, mW, mZ)

In [7]:
ROOT.ildStyle.SetOptStat(1)
# ROOT.ildStyle.SetOptStat(0)

In [8]:
%%cpp
using namespace ROOT::VecOps;

In [9]:
# n_points = 5000000
n_points = 1000000

df = ROOT.RDataFrame(n_points)

In [10]:
df = df.Define("co", "gRandom->Uniform(-1., 1.)")
df = df.Define("co1", "gRandom->Uniform(-1., 1.)")
df = df.Define("co2", "gRandom->Uniform(-1., 1.)")
df = df.Define("ph1", "gRandom->Uniform(-1.*ROOT::Math::Pi(), 1.*ROOT::Math::Pi())")
df = df.Define("ph2", "gRandom->Uniform(-1.*ROOT::Math::Pi(), 1.*ROOT::Math::Pi())")

In [11]:
df = df.Define("w_proba", "OOTools::proba(co, co1, co2, ph1, ph2, 0, true, 0)")
# FIXME: do wraparound for phi1, phi2, using ROOT:TVector2::Phi_mpi_pi(x)
df = df.Define("w_proba_cp", "OOTools::proba(co, -co2, -co1, ROOT::Math::Pi() + ph2, ROOT::Math::Pi() + ph1, 0, true, 0)")
df = df.Define("w_proba_fz1", "OOTools::proba(co, co1, co2, ph1, ph2, 1, false, 1)")
df = df.Define("w_proba_diff", "w_proba - w_proba_cp")

In [12]:
for var in [f"fg{i}" for i in range(1,8)] + [f"fz{i}" for i in range(1,8)]:
    df = df.Define(var, f"OOTools::{var}(co, co1, co2, ph1, ph2)")

In [13]:
f_vars = ["fg1", "fz1", "fg2", "fz2", "fg3", "fz3", "fg5", "fz5"]
columns = df.AsNumpy(f_vars+ ["w_proba", "w_proba_fz1"])

In [14]:
f_vars_a = np.asarray([columns[v] for v in f_vars])

In [15]:
# w_proba_norm = columns["w_proba"] / np.max(columns["w_proba"])
# w_proba_fz1_norm = columns["w_proba_fz1"] / np.max(columns["w_proba"])
# normalized such that we have frequencies for the SM values
w_proba_norm = (columns["w_proba"] / np.min(columns["w_proba"])).astype(int)
w_proba_fz1_norm = columns["w_proba_fz1"] / np.min(columns["w_proba"])

In [16]:
# ugh aweights and fweights are not equivalent :(
# f_vars_cov = np.cov(f_vars_a, ddof=0, aweights=w_proba_norm)
f_vars_cov = np.cov(f_vars_a, ddof=0, fweights=w_proba_norm)
# print(np.round(1.e3 * f_vars_cov, decimals=2))
print(np.round(1.e3 * f_vars_cov, decimals=0))
# meh this c is wrong :(
# even when going to fweights :(
# oops I did this at 250 GeV not 190 :(
# now it is correct!!!

[[ 13.  17. -13. -14. -21. -21.   1.   0.]
 [ 17.  27. -15. -23. -23. -37.   0.   0.]
 [-13. -15.  17.  19.  28.  28.  -1.   0.]
 [-14. -23.  19.  32.  30.  51.  -0.   0.]
 [-21. -23.  28.  30.  66.  67.  -2.   4.]
 [-21. -37.  28.  51.  67. 122.  -1.   1.]
 [  1.   0.  -1.  -0.  -2.  -1.   5.   8.]
 [  0.   0.   0.   0.   4.   1.   8.  17.]]


In [17]:
diffs = [0.] * len(f_vars)

for i, f in enumerate(f_vars):
    mean_1 = np.average(columns[f], weights=0.1*w_proba_fz1_norm)
    mean_0 = np.average(columns[f], weights=w_proba_norm)
    diffs[i] = mean_1 - mean_0

g_vec = np.linalg.solve(f_vars_cov, np.asarray(diffs))
print(g_vec)

[-1.37350700e-01  7.64869268e+01  9.43891441e-02 -3.84515227e-02
 -1.04859467e-01  7.06669612e-02 -3.53733918e-01  1.16740138e-01]


In [18]:
pi = ROOT.Math.Pi()

# or just fill 1D histograms with weight proba??
# p_co = df.Profile1D(("", ";co", 25, -1., 1.), "co", "proba")
# p_co1 = df.Profile1D(("", ";co1", 25, -1., 1.), "co1", "proba")
# p_co2 = df.Profile1D(("", ";co2", 25, -1., 1.), "co2", "proba")
# p_ph1 = df.Profile1D(("", ";ph1", 25, -pi, pi), "ph1", "proba")
# p_ph2 = df.Profile1D(("", ";ph2", 25, -pi, pi), "ph2", "proba")
h_co  = df.Histo1D(("", ";co", 25, -1., 1.), "co", "w_proba")
h_co1 = df.Histo1D(("", ";co1", 25, -1., 1.), "co1", "w_proba")
h_co2 = df.Histo1D(("", ";co2", 25, -1., 1.), "co2", "w_proba")
h_ph1 = df.Histo1D(("", ";ph1", 25, -pi, pi), "ph1", "w_proba")
h_ph2 = df.Histo1D(("", ";ph2", 25, -pi, pi), "ph2", "w_proba")

h_weight = df.Histo1D("w_proba")

h_fg1 = df.Histo1D(("", ";fg1", 100, -1.5, 1.5), "fg1", "w_proba")
h_fg2 = df.Histo1D(("", ";fg2", 100, -1.5, 1.5), "fg2", "w_proba")
h_fg3 = df.Histo1D(("", ";fg3", 100, -1.5, 1.5), "fg3", "w_proba")
h_fg4 = df.Histo1D(("", ";fg4", 100, -1.5, 1.5), "fg4", "w_proba")
h_fg5 = df.Histo1D(("", ";fg5", 100, -1.5, 1.5), "fg5", "w_proba")

h_fz1 = df.Histo1D(("", ";fz1", 100, -1.5, 1.5), "fg1", "w_proba")
h_fz2 = df.Histo1D(("", ";fz2", 100, -1.5, 1.5), "fg2", "w_proba")
h_fz3 = df.Histo1D(("", ";fz3", 100, -1.5, 1.5), "fg3", "w_proba")
h_fz4 = df.Histo1D(("", ";fz4", 100, -1.5, 1.5), "fg4", "w_proba")
h_fz5 = df.Histo1D(("", ";fz5", 100, -1.5, 1.5), "fg5", "w_proba")

h_w_proba_diff = df.Histo1D(("", "; standard - CP", 51, -1e-13, 1e-13), "w_proba_diff")

In [19]:
df.Display(("w_proba", "w_proba_cp")).Print()

+-----+-----------+------------+
| Row | w_proba   | w_proba_cp | 
+-----+-----------+------------+
| 0   | 43.485135 | 43.485135  | 
+-----+-----------+------------+
| 1   | 5.850257  | 5.850257   | 
+-----+-----------+------------+
| 2   | 55.733652 | 55.733652  | 
+-----+-----------+------------+
| 3   | 9.872475  | 9.872475   | 
+-----+-----------+------------+
| 4   | 12.349686 | 12.349686  | 
+-----+-----------+------------+


In [20]:
c_co = ROOT.TCanvas()
# p_co.SetMinimum(0)
# p_co.Draw()
h_co.SetMinimum(0)
h_co.Draw()
c_co.Draw()

c_co1 = ROOT.TCanvas()
# p_co1.SetMinimum(0)
# p_co1.Draw()
h_co1.SetMinimum(0)
h_co1.Draw()
c_co1.Draw()

c_ph1 = ROOT.TCanvas()
# p_ph1.SetMinimum(0)
# p_ph1.Draw()
h_ph1.SetMinimum(0)
h_ph1.Draw()
c_ph1.Draw()

c_co2 = ROOT.TCanvas()
# p_co2.SetMinimum(0)
# p_co2.Draw()
h_co2.SetMinimum(0)
h_co2.Draw()
c_co2.Draw()

c_ph2 = ROOT.TCanvas()
# p_ph2.SetMinimum(0)
# p_ph2.Draw()
h_ph2.SetMinimum(0)
h_ph2.Draw()
c_ph2.Draw()

c_weight = ROOT.TCanvas()
h_weight.SetMinimum(0)
h_weight.Draw()
c_weight.Draw()

c_w_proba_diff = ROOT.TCanvas()
h_w_proba_diff.Draw()
c_w_proba_diff.Draw()


In [21]:

c_fg1 = ROOT.TCanvas()
h_fg1.SetMinimum(0)
h_fg1.Draw()
c_fg1.Draw()

c_fg2 = ROOT.TCanvas()
h_fg2.SetMinimum(0)
h_fg2.Draw()
c_fg2.Draw()

c_fg3 = ROOT.TCanvas()
h_fg3.SetMinimum(0)
h_fg3.Draw()
c_fg3.Draw()

c_fg4 = ROOT.TCanvas()
h_fg4.SetMinimum(0)
h_fg4.Draw()
c_fg4.Draw()

c_fg5 = ROOT.TCanvas()
h_fg5.SetMinimum(0)
h_fg5.Draw()
c_fg5.Draw()

c_fz1 = ROOT.TCanvas()
h_fz1.SetMinimum(0)
h_fz1.Draw()
c_fz1.Draw()

c_fz2 = ROOT.TCanvas()
h_fz2.SetMinimum(0)
h_fz2.Draw()
c_fz2.Draw()

c_fz3 = ROOT.TCanvas()
h_fz3.SetMinimum(0)
h_fz3.Draw()
c_fz3.Draw()

c_fz4 = ROOT.TCanvas()
h_fz4.SetMinimum(0)
h_fz4.Draw()
c_fz4.Draw()

c_fz5 = ROOT.TCanvas()
h_fz5.SetMinimum(0)
h_fz5.Draw()
c_fz5.Draw()